In [27]:
import pandas as pd
import numpy as np
video_data = pd.read_csv('Screenshots Data - Sheet1.csv')
video_data

,User,Type,Screenshot,Order,Video,Channels,Verified,BiasRating,Factual,MBFC Credibility Rating
0,Cole,Control,Search,1,2020 Election Results and Analysis | TODAY,TODAY,NaN,left-center,mostly factual,NaN
1,Cole,Control,Search,2,Election Results 2020: Last Week Tonight with ...,LastWeekTonight,NaN,NaN,NaN,NaN
2,Cole,Control,Search,3,LIVE: Latest News Headlines and Events | ABC N...,ABC News,NaN,left-center,high,high
3,Cole,Control,Search,4,"John King: If Trump loses here, he's done",CNN,NaN,left,mixed,medium
4,Cole,Control,Search,5,2020 Election Results and Analysis | TODAY,TODAY,NaN,left-center,mostly factual,NaN
...,...,...,...,...,...,...,...,...,...,...
324,Dongxue,Right,R7,2,Dogfights: F-86 Sabres Battle at Extreme Speed...,history,NaN,NaN,NaN,NaN
325,Dongxue,Right,R7,3,Pete Hegseth responds to Ilhan Omar's tweet ab...,fox news,NaN,right,mixed,medium credibility
326,Dongxue,Right,R7,4,fremont police release body cam video of deadl...,mercury news,NaN,left-center,high,NaN
327,Dongxue,Right,R7,5,desantis hits ocasio-cortez: i don't care if s...,fox news,NaN,right,mixed,medium credibility


In [23]:
df = video_data[["User", "Type", "Screenshot", "Order", "Video", "Channels", "BiasRating"]]

In [24]:
df

,User,Type,Screenshot,Order,Video,Channels,BiasRating
0,Cole,Control,Search,1,2020 Election Results and Analysis | TODAY,TODAY,left-center
1,Cole,Control,Search,2,Election Results 2020: Last Week Tonight with ...,LastWeekTonight,NaN
2,Cole,Control,Search,3,LIVE: Latest News Headlines and Events | ABC N...,ABC News,left-center
3,Cole,Control,Search,4,"John King: If Trump loses here, he's done",CNN,left
4,Cole,Control,Search,5,2020 Election Results and Analysis | TODAY,TODAY,left-center
...,...,...,...,...,...,...,...
324,Dongxue,Right,R7,2,Dogfights: F-86 Sabres Battle at Extreme Speed...,history,NaN
325,Dongxue,Right,R7,3,Pete Hegseth responds to Ilhan Omar's tweet ab...,fox news,right
326,Dongxue,Right,R7,4,fremont police release body cam video of deadl...,mercury news,left-center
327,Dongxue,Right,R7,5,desantis hits ocasio-cortez: i don't care if s...,fox news,right


In [31]:
users = ["Benje", "Candice", "Derren", "Dongxue", "Cole", "Jaeden"]
rounds = ["Search", "R1", "R2", "R3", "R4", "R5", "R6", "R7"]
json_obj = {"users": []}
switch = 1
for index1, user in enumerate(users):
    user = {"column_user_id": int(not(switch)), "rounds": []}
    switch = int(not(switch))
    # print(user)
    r = 0
    for rnd in range(len(rounds)):
        single_rnd = {"round_id": r, "videos": []}
        r += 1
        extracted_df = df.loc[(df['User'] == users[index1]) & (df["Screenshot"] == rounds[rnd])]
        # print(f'extracted_df: {extracted_df}')
        null = None
        video_array = eval(extracted_df.loc[:, ["Video", "Channels", "BiasRating"]].to_json(orient = "records"))
        single_rnd["videos"] = video_array
        user["rounds"].append(single_rnd)
    json_obj["users"].append(user)

# json_obj


In [32]:
import json
with open('../frontend/src/watch-recommendations.json', 'w') as outfile:
    json.dump(json_obj, outfile, indent=3)


In [26]:
video_data = pd.read_csv('data_new.csv')

df2 = video_data[["Column", "UserID", "User", "Type", "Screenshot", "Order", "Video", "Channels", "Bias Rating", "Factual"]]
users = ["Benje", "Candice", "Derren", "Dongxue", "Cole", "Jaeden"]
rounds = ["Search", "R1", "R2", "R3", "R4", "R5", "R6", "R7"]

df2.loc[(df2['Column'] == 0) & (df2['UserID'] == 0)]

import numpy as np
captions = []
for c in range(3):
    column = []
    for u in range(2):
        user = []
        for r in range(1, len(rounds)):
            extracted_df = df2.loc[(df2['Column'] == c) & (df2['UserID'] == u) & (df2["Screenshot"] == rounds[r])]
            x = extracted_df["Bias Rating"].to_string(index=False).strip().replace('\n', ', ')
            user.append([x])
        column.append(user)
    captions.append(column)

captions


In [26]:
cols, users, rounds, blah = captions.shape
colors = {
    'least bias': "#f1e15b",
    'left': "#61cdbb",
    'left-center': "#e8c1a0",
    'NaN': "#f47560",
    'right': "#e8a838",
    'right-center': "#9467bd",
    }

preprocessed_data = []
for col in range(cols):
    c = []
    for user in range(users):
        u = []
        for r in range(rounds):
            # print(f'col: {col}, user: {user}, round: {r}')
            curr_data = np.array(captions[col][user][r])
            curr_data = curr_data[0].split(',')
            for i in range(len(curr_data)):
                curr_data[i] = curr_data[i].strip()

            (unique, counts) = np.unique(curr_data, return_counts=True)
            freq = np.asarray((unique,counts)).T
            round_data = []
            for f in freq:
                res = {}

                if f[0] == "NaN":
                    res["id"] = "Unrated"
                    res["label"] = "Unrated"
                else:
                    res["id"] = f[0].title()
                    res["label"] = f[0].title()

                res["value"] = int(f[1])
                # res["color"] = colors[f[0]]

                round_data.append(res)

            u.append(round_data)
        c.append(u)
    preprocessed_data.append(c)

print(preprocessed_data)